# Experiments II
In this notebook we will start applying models on our emotion-detection data.

packages

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import plotly.express as px
import nltk
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score
import tensorflow as tf

data

In [29]:
#Importing data
data = pd.read_excel ( '../Datasets/emotions_detection_datasets/02_final_data.xlsx' )
#data.drop (columns = ['Unnamed: 0'], inplace = True)
data

,text,emotion,language
0,الاوليمبياد الجايه هكون لسه ف الكليه ..,none,egy
1,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,anger,egy
2,كتنا نيله ف حظنا الهباب xD,sadness,egy
3,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,joy,egy
4,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,none,egy
...,...,...,...
454301,كيف يمكن لهذه الأمور أن تتغير فجأة وبدون سابق ...,surprised,arb_classic
454302,كانت المفاجأة كبيرة جداً ولم أتمكن من استيعابه...,surprised,arb_classic
454303,لم أكن أتوقع هذا التغيير السريع في الأحداث.,surprised,arb_classic
454304,شعرت بالدهشة الكبيرة عندما رأيت النتيجة المفاجئة.,surprised,arb_classic


Exploratory Data Analysis

In [30]:
print ('number of rows', data.shape[0])

number of rows 454306


In [31]:
#Languages
print ('number of rows per language')
data['language'].value_counts()

number of rows per language


language
eng            416809
egy             10065
arb-trans        8000
arb              6369
fr               5000
ar               4582
alg-latin        2400
arb_classic      1081
Name: count, dtype: int64

In [32]:
#Emotions
print ('number of rows per emotion')
data['emotion'].value_counts()

number of rows per emotion


emotion
joy             146674
sadness         126615
anger            60741
fear             51456
love             37048
surprise         16698
sad               2200
angry             2177
happy             1816
none              1550
sympathy          1062
hapiness           966
Neutral            925
Angry              469
neutral            390
surprised          290
Laughing           275
trust              252
disgust            213
Wondering          207
Surprised          193
Sad                187
Happy              144
anticipation        12
Name: count, dtype: int64

In order to orgnise emotions in our data, and due to the limited number of rows for some emotions we are going to do the following modifications:

[sadness = sad = Sad]

[joy = happiness = fun = Laughing = Happy = love = enthusiasm = relief]

[none = neutral = Neutral = boredom = empty = sympathy = wondering]

[Angry = anger = hate]

[Surprised = surprise]

[worry = fear]

[s, lov, lo, j] to be deleted

In [33]:
#[s, lov, lo, j] to be deleted
idx_to_delete = data[data['emotion'].isin (['s', 'lov', 'lo', 'j'])].index
data_01 = data.drop (index= idx_to_delete)
data_01

,text,emotion,language
0,الاوليمبياد الجايه هكون لسه ف الكليه ..,none,egy
1,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,anger,egy
2,كتنا نيله ف حظنا الهباب xD,sadness,egy
3,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,joy,egy
4,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,none,egy
...,...,...,...
454301,كيف يمكن لهذه الأمور أن تتغير فجأة وبدون سابق ...,surprised,arb_classic
454302,كانت المفاجأة كبيرة جداً ولم أتمكن من استيعابه...,surprised,arb_classic
454303,لم أكن أتوقع هذا التغيير السريع في الأحداث.,surprised,arb_classic
454304,شعرت بالدهشة الكبيرة عندما رأيت النتيجة المفاجئة.,surprised,arb_classic


In [34]:
#Doing the emotional mapping, emotion to their original form
emotion_mapping = {'sadness':'sad', 'Sad': 'sad',
                    'joy': 'happy', 'happiness': 'happy', 'hapiness': 'happy', 'fun': 'happy', 'Laughing': 'happy', 'Happy': 'happy', 'love': 'happy', 'enthusiasm': 'happy', 'relief': 'happy', 'trust': 'happy',
                    'none': 'neutral', 'Neutral': 'neutral', 'boredom': 'neutral', 'empty': 'neutral', 'sympathy': 'neutral', 'Wondering': 'neutral', 'anticipation': 'neutral',
                    'Angry': 'angry', 'anger': 'angry', 'hate': 'angry', 'disgust': 'angry',
                    'Surprised': 'surprised', 'surprise': 'surprised',
                    'worry': 'fear', 'fear': 'fear'}
#Remaining emotions : ['sad', 'happy', 'surprised', 'fear', 'angry', neutral]
data_01['emotion'] = data_01['emotion'].replace (emotion_mapping)

In [35]:
#Number of rows per emotion
px.bar (data_01['emotion'].value_counts(), width = 500, title = 'Number of rows per emotion')

We end up with 6 emotions : happy, surprised, neutral, fear, sad,  angry

Some rows in text feature contains some usernames that start with: @
These usernames are not gonna add any information to our models.
In contrast they might make our model biased.
Due to the reasons above we should delete these usernames

In [ ]:
#Defining delete_username function
def delete_username (sentence):
    """
    This function takes as input a sentence and delete all the words in it that start with the caracter @
    sentence: str
    exemple : '@Hamza I don't agree with this politic.' ----> 'I don't agree with this politic'
    """
    #Get tokens from sentence
    tokens = nltk.word_tokenize (sentence)
    #initiate a list that will contains the remaning tokens
    final_tokens = []
    #Loop over these tokens
    i = 0
    while i < len (tokens):
        #See if the token is starting with a @
        if tokens[i].startswith("@"):
            #if so skip it and the next word
            i = i+2
        else:
            #The word doesn't start with @ so add it
            final_tokens.append (tokens[i])
            i = i+1

    #Finally concatenate tokens to a sentence
    sentence = ' '.join (final_tokens)
    return sentence

#Testing the function
sentence = 'Hello @Hamza how are you doing ?, did you see @mohamed I want to talk with him'
delete_username(sentence)


###  Applying models on the dataset before fine-tuning and evaluating the performance

In [ ]:
#Importing the model
from transformers import AutoTokenizer, XLMRobertaForSequenceClassification, pipeline
import torch

In [ ]:
#Model
model_name = 'upsalite/xlm-roberta-base-finetuned-emotion-37-labels'
tokenizer = AutoTokenizer.from_pretrained (model_name)
roberta_model = XLMRobertaForSequenceClassification.from_pretrained (model_name)

In [ ]:
#Function that returns the emotions with thier probabilities using the model predictions
def get_emotions_probabilities (text, model, tokenizer):
    """This function returns each emotion with its probabilite
    from output of a model roberta in this case, and a text
    text : input text
    tokenizer : tokenizer to tokenize text
    model: roberta-model for inference
    """

    text_tokenized = tokenizer (text, return_tensors = 'pt')
    with torch.no_grad ():
        output = roberta_model ( **text_tokenized )
    predictions = torch.nn.functional.softmax (output.logits, dim=-1)[0]
    emotions = model.config.id2label
    emotion_probabilitie = {}
    for i, item in enumerate (predictions) :
        emotion_probabilitie[emotions[i]] = item.item()

    return emotion_probabilitie

> Available fined tuned models on emotion datasets:

daveni/twitter-xlm-roberta-emotion-es   F1:71%

SamLowe/roberta-base-go_emotions    F1: 41%

upsalite/xlm-roberta-base-finetuned-emotion-37-labels  F1: 71%

02shanky/finetuned-twitter-xlm-roberta-base-emotion  F1: 93%

In [ ]:
#Models :
model = pipeline ( task = 'text-classification', model = '02shanky/finetuned-twitter-xlm-roberta-base-emotion' )

In [ ]:
#Test in french language
sample_text = """ ce que tu as fait est inacceptable, """
model (sample_text)

In [ ]:
#Test in egy language
sample_text = data_01 [data['language'] == 'egy'][10063:10064]['text'].item()
#'وعليك قبلنا يانجم النجوم ياعندليب الحب والاحساس'
model (sample_text)

In [ ]:
#Test in alg latin
sample_text = 'hadi khabar zwina bzaf, ana far7an'
model (sample_text)

the "02shanky/finetuned-twitter-xlm-roberta-base-emotion" model doesn't support alg latin

Choosing a subset of the data to test the models on

#### `english data`

In [ ]:
#Subset of data (1000 rows)
eng_data = data_01 [ data_01['language'] == 'eng' ]
eng_data = eng_data[0: 1000]
eng_data

In [ ]:
# df to array
eng_text = eng_data ['text'].to_list()

In [ ]:
#Applying the model
eng_output = model ( eng_text )
eng_output

The list of emotion reconised by model is : love, joy, anger, sadness, surprise, fear

While emotions in our usecase are: happy ,neutral, angry, sad, surprised, fear   

That means we should do the necessay mapping before calculating the accuracy

In [ ]:
#emotions dictionnary
emotion_mapping = {'love': 'happy', 'joy': 'happy', 'anger': 'angry', 'sadness': 'sad', 'surprise': 'surprised', 'fear': 'fear'}

In [ ]:
#Getting the labels from eng_output
eng_output_labels = []
for line in eng_output:
    #gitting the model emotion
    emotion = emotion_mapping [line['label']]
    #transfoming it to original form
    eng_output_labels.append (emotion)

In [ ]:
#Mesuring the metrics score
print ('the accuracy score of the model for english data is :', accuracy_score ( eng_data['emotion'], eng_output_labels ))
print ('the f1 score of the model for english data is :', f1_score ( eng_data['emotion'], eng_output_labels, average = 'macro' ))
print ('the precision score of the model for english data is :', precision_score ( eng_data['emotion'], eng_output_labels, average = 'macro' ))
print ('the recall score of the model for english data is :', recall_score ( eng_data['emotion'], eng_output_labels, average = 'macro' ))

In [ ]:
#Visualising the confusion matrix
matrix = confusion_matrix ( eng_data['emotion'], eng_output_labels )
labels = ['angry', 'fear', 'happy', 'neutral', 'sad', 'surprised'  ]
seaborn.heatmap ( matrix, annot=True, cmap='Blues', fmt='d', xticklabels= labels, yticklabels= labels )

The model perform great on the english data

#### ` French data `

In [ ]:
#Subset of data (1000 rows)
fr_data = data_01 [ data_01['language'] == 'fr' ]
fr_data = fr_data[0: 1000]

# df to array
fr_text = fr_data ['text'].to_list()
#Applying the model
fr_output = model ( fr_text )

#emotions dictionnary
emotion_mapping = {'love': 'happy', 'joy': 'happy', 'anger': 'angry', 'sadness': 'sad', 'surprise': 'surprised', 'fear': 'fear'}
#Getting the labels from eng_output
fr_output_labels = []
for line in fr_output:
    #gitting the model emotion
    emotion = emotion_mapping [line['label']]
    #transfoming it to original form
    fr_output_labels.append (emotion)
#Mesuring the metrics score
print ('the accuracy score of the model for french data is :', accuracy_score ( fr_data['emotion'], fr_output_labels ))
print ('the f1 score of the model for french data is :', f1_score ( fr_data['emotion'], fr_output_labels, average = 'macro' ))
print ('the precision score of the model for french data is :', precision_score ( fr_data['emotion'], fr_output_labels, average = 'macro' ))
print ('the recall score of the model for french data is :', recall_score ( fr_data['emotion'], fr_output_labels, average = 'macro' ))
#Visualising the confusion matrix
matrix = confusion_matrix ( fr_data['emotion'], fr_output_labels )
labels = ['angry', 'fear', 'happy', 'neutral', 'sad', 'surprised'  ]
seaborn.heatmap ( matrix, annot=True, cmap='Blues', fmt='d', xticklabels= labels, yticklabels= labels )

#### `Egyptian data`

In [ ]:
#Subset of data (1000 rows)
egy_data = data_01 [ data_01['language'] == 'egy' ]
egy_data = egy_data[0: 1000]
# df to array
egy_text = egy_data ['text'].to_list()
#Applying the model
egy_output = model ( egy_text )

#emotions dictionnary
emotion_mapping = {'love': 'happy', 'joy': 'happy', 'anger': 'angry', 'sadness': 'sad', 'surprise': 'surprised', 'fear': 'fear'}
#Getting the labels from output
egy_output_labels = []
for line in egy_output:
    #gitting the model emotion
    emotion = emotion_mapping [line['label']]
    #transfoming it to original form
    egy_output_labels.append (emotion)
#Mesuring the metrics score
print ('the accuracy score of the model for egyptian data is :', accuracy_score ( egy_data['emotion'], egy_output_labels ))
print ('the f1 score of the model for egyptian data is :', f1_score ( egy_data['emotion'], egy_output_labels, average = 'macro' ))
print ('the precision score of the model for egyptian data is :', precision_score ( egy_data['emotion'], egy_output_labels, average = 'macro' ))
print ('the recall score of the model for egyptian data is :', recall_score ( egy_data['emotion'], egy_output_labels, average = 'macro' ))

#Visualising the confusion matrix
matrix = confusion_matrix ( egy_data['emotion'], egy_output_labels )
labels = ['angry', 'fear', 'happy', 'neutral', 'sad', 'surprised'  ]
seaborn.heatmap ( matrix, annot=True, cmap='Blues', fmt='d', xticklabels= labels, yticklabels= labels )

#### `Arab classic data`

The model can't generate inference on text that length is bigger then 900. That why we fixed the max_text_length to 900.

In [ ]:
#Max text length
max_text_length = 600

In [ ]:
#Subset of data (1000 rows)
#First lets shuflle data
from sklearn.utils import shuffle
arb_data = data_01 [ data_01['language'] == 'arb' ]
arb_data = shuffle ( arb_data )
arb_data = arb_data[0: 1000]
# df to array
arb_text = arb_data ['text'].to_list()
#Limiting the size of texts in arb_text to max_text_length
arb_text_lim = []
for text in arb_text:
    arb_text_lim.append  (text [0: max_text_length])

In [ ]:
#Nmb of emotions in arabic data
print ('Nmb of emotions in arabic data is:')
arb_data.emotion.value_counts()

In [ ]:
#Applying the model
arb_output = model ( arb_text_lim )

#emotions dictionnary
emotion_mapping = {'love': 'happy', 'joy': 'happy', 'anger': 'angry', 'sadness': 'sad', 'surprise': 'surprised', 'fear': 'fear'}
#Getting the labels from output
arb_output_labels = []
for line in arb_output:
    #gitting the model emotion
    emotion = emotion_mapping [line['label']]
    #transfoming it to original form
    arb_output_labels.append (emotion)
#Mesuring the metrics score
print ('the accuracy score of the model for arab data is :', accuracy_score ( arb_data['emotion'], arb_output_labels ))
print ('the f1 score of the model for arab data is :', f1_score ( arb_data['emotion'], arb_output_labels, average = 'macro' ))
print ('the precision score of the model for arab data is :', precision_score ( arb_data['emotion'], arb_output_labels, average = 'macro' ))
print ('the recall score of the model for arab data is :', recall_score ( arb_data['emotion'], arb_output_labels, average = 'macro' ))
#Visualising the confusion matrix
matrix = confusion_matrix ( arb_data['emotion'], arb_output_labels )
labels = ['angry', 'fear', 'happy', 'sad' ]
seaborn.heatmap ( matrix, annot=True, cmap='Blues', fmt='d', xticklabels= labels, yticklabels= labels )

#### `Darija written in latin data`

In [ ]:

#Subset of data (1000 rows)
drj_data = data_01 [data_01['language'] == 'alg-latin']
drj_data = shuffle ( drj_data )
drj_data = drj_data[0: 1000]
# df to array
drj_text = drj_data ['text'].to_list()
#Applying the model
drj_output = model ( drj_text )

#emotions dictionnary
emotion_mapping = {'love': 'happy', 'joy': 'happy', 'anger': 'angry', 'sadness': 'sad', 'surprise': 'surprised', 'fear': 'fear'}
#Getting the labels from output
drj_output_labels = []
for line in drj_output:
    #gitting the model emotion
    emotion = emotion_mapping [line['label']]
    #transfoming it to original form
    drj_output_labels.append (emotion)
#Mesuring the metrics score
print ('the accuracy score of the model for darija data is :', accuracy_score ( drj_data['emotion'], drj_output_labels ))
print ('the f1 score of the model for darija data is :', f1_score ( drj_data['emotion'], drj_output_labels, average = 'macro' ))
print ('the precision score of the model for darija data is :', precision_score ( drj_data['emotion'], drj_output_labels, average = 'macro' ))
print ('the recall score of the model for darija data is :', recall_score ( drj_data['emotion'], drj_output_labels, average = 'macro' ))
#Visualising the confusion matrix
matrix = confusion_matrix ( drj_data['emotion'], drj_output_labels )
labels = ['angry', 'fear', 'happy', 'neutral', 'sad', 'surprised'  ]
seaborn.heatmap ( matrix, annot=True, cmap='Blues', fmt='d', xticklabels= labels, yticklabels= labels )

Le modèle est très faible en darija

Récapitulatif des performances du modèle {finetuned-twitter-xlm-roberta-base-emotion} en détection des émotions en différentes langues  :

|       | F1-score      | Accuracy      | Precision | Recall    |
-------- | ------------ |  ----------------- | -----| --------
|English |     0.9374479  |   0.972   | 0.938283 | 0.93754  |
| French | 0.62124  |  0.708 | 0.6427| 0.616444 |
|Egyptain | 0.377739 | 0.456 | 0.4133 | 0.4557 |
|Arabic | 0.2353 | 0.471 | nan  | nan |
|Darija | 0.12755 | 0.211 | 0.1969 |  0.20563 |

You need to add, type of data, sample of data

#### `Darija-Lattin with DarijaBert part II`

### Model fine-tuning on our datasets

In this part of the notebook we will fine tune the model {02shanky/finetuned-twitter-xlm-roberta-base-emotion} on datasets where he performed baddly such as French, Arabic, Darija ...

#### 1st Approche using Pytorch

In [ ]:
#Packages
from transformers import Trainer, TrainingArguments, AutoTokenizer, XLMRobertaForSequenceClassification, XLMRobertaTokenizer
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
#Let's encode first the emotion column
emotion_to_num = {"happy": 0, "sad": 1, "angry": 2, "fear": 3, "surprised": 4}
fr_train_dataset['emotion'] = [emotion_to_num[emotion] for emotion in fr_train_dataset['emotion']]
fr_test_dataset['emotion'] = [emotion_to_num[emotion] for emotion in fr_test_dataset['emotion']]

In [ ]:
#Method 1 of customizing data

#Customizing dataset
class CustomDataset(Dataset):
    def __init__(self, texts, emotions, tokenizer, max_length):
        self.texts = texts
        self.emotions = emotions
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        texts = self.texts[idx]
        emotions = self.emotions[idx]
        # Tokenize the text
        encoding = self.tokenizer(texts, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(emotions, dtype=torch.long)
        }
#Applying Custom data on our data
tokenizer = XLMRobertaTokenizer.from_pretrained (model_name)
max_length = max(len(tokenizer.encode(text)) for text in fr_train_dataset['text'])
fr_train_dataset_ = CustomDataset (fr_train_dataset['text'], fr_train_dataset['emotion'], tokenizer, max_length)
fr_test_dataset_ = CustomDataset (fr_test_dataset['text'], fr_test_dataset['emotion'], tokenizer, max_length)


In [ ]:
#Method 2 of customizing data
"""
#Tokenizing
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(fr_train_dataset['text'].to_list(), truncation=True, padding='max_length', max_length=128)
test_encodings = tokenizer(fr_test_dataset['text'].to_list(), truncation=True, padding='max_length', max_length=128)

#Customizing data
class CustomDataset(Dataset):
    def __init__(self, encodings, emotion):
        self.encodings = encodings
        self.emotion = emotion

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['emotion'] = torch.tensor(self.emotion[idx])
        return item

    def __len__(self):
        return len(self.emotion)

fr_train_dataset_ = CustomDataset(train_encodings, fr_train_dataset['emotion'])
fr_test_dataset_ = CustomDataset(test_encodings, fr_test_dataset['emotion'])
"""

In [ ]:
#Model
model_name = "02shanky/finetuned-twitter-xlm-roberta-base-emotion"
model = XLMRobertaForSequenceClassification.from_pretrained ( model_name )
#Let's freeze the model pre-trained weights
for param in model.base_model.parameters():
    param.requires_grad = False

In [ ]:
#Let's add a classifier layer to the model
num_emotions = fr_train_dataset['emotion'].value_counts ().count() #5
model.num_labels= num_emotions
model.classifier = torch.nn.Linear (model.config.hidden_size, num_emotions )

In [ ]:
# Specify more parameters
# model.classifier = torch.nn.Sequential(
#     torch.nn.Linear(model.config.hidden_size, hidden_layer_size),
#     torch.nn.ReLU(),
#     torch.nn.Dropout(dropout_prob),
#     torch.nn.Linear(hidden_layer_size, num_labels)
# )

In [ ]:
#Trianing parameters
training_args = TrainingArguments (
    per_device_train_batch_size= 1976,
    num_train_epochs= 1,
    evaluation_strategy= "epoch",
    logging_dir="./log",
    output_dir= "./output"
 )

In [ ]:
#Our model doesn't return loss, so we need to specify to the trained how can he compute the loss
class MyTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = self.loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
#Initiating the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=fr_train_dataset_,
    eval_dataset=fr_test_dataset_,
    #compute_metrics=None  # If you don't need metrics during training
)
#Training the model
trainer.train ()

In [ ]:
# Évaluation du modèle
trainer.evaluate()


#### 2end Approch using Tensorflow  

In [ ]:
#Packages
from transformers import TFXLMRobertaForSequenceClassification, XLMRobertaTokenizer

In [ ]:
#Spliting data
data = data_01 [data_01['language'].isin (["arb", "ar", "egy", "arb_classic"])]
#Choosing only 3000 rows of emotion happy
no_happy = data [ data.emotion.isin ( ["happy"] ) == False ]
happy = data [ data.emotion.isin ( ["happy"] ) ][0: 3200]
#Additional sad data 
arb_trans = data_01 [data_01['language'].isin (["arb-trans"])]
sad = arb_trans[ arb_trans.emotion.isin ( ['sad'] ) ][0: 1000]
data = pd.concat ( [happy, no_happy, sad] )
data = data.reset_index (drop= True)
#data = data.drop ( columns = ['language'] )
#Drop null values and columns with neutral as emotion
data.dropna (inplace = True)
index_to_delete = data[data['emotion'] == "neutral"].index
data.drop ( index = index_to_delete, inplace = True )
#drop duplicates values of column text
data = data.drop_duplicates ( subset = "text" )
train_dataset, test_dataset = train_test_split (data, test_size = 0.074, shuffle=True, random_state=42)
print ( "train_dataset size : ", train_dataset.shape )
print ( "test_dataset size : ", test_dataset.shape )

In [ ]:
# Load model directly
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("hatemnoaman/bert-base-arabic-finetuned-emotion")
tf_model = TFAutoModelForSequenceClassification.from_pretrained("hatemnoaman/bert-base-arabic-finetuned-emotion", from_pt = True)

tf_model.layers[0].trainable = False 
tf_model.summary ()

In [ ]:
# Extracting Model and Tokenizer
model_name = "02shanky/finetuned-twitter-xlm-roberta-base-emotion"
tokenizer = XLMRobertaTokenizer.from_pretrained (model_name)
tf_model = TFXLMRobertaForSequenceClassification.from_pretrained (model_name, from_pt = True)

tf_model.layers[0].trainable = False

#Model architecture
#tf_model.summary ()

In [ ]:
#Tokenizing the dataset
max_length = 500
train_inputs = tokenizer(train_dataset['text'].tolist(), padding=True, truncation=True, max_length = max_length, return_tensors="tf")
#val_inputs = tokenizer(val_dataset['text'].tolist(), padding=True, truncation=True, max_length = max_length, return_tensors="tf")
test_inputs = tokenizer(test_dataset['text'].tolist(), padding=True, truncation=True, max_length = max_length, return_tensors="tf")

#Let's encode the emotion column [emotion <--> label]
emotion_to_num = {"happy": 0, "sad": 1, "angry": 2, "fear": 3, "surprised": 4, 'neutral': 5}
num_to_emotion = {0: "happy", 1: "sad", 2: "angry", 3: "fear", 4: "surprised", 5: "neutral"}
train_labels = [emotion_to_num[emotion] for emotion in train_dataset['emotion']]
#val_labels = [emotion_to_num[emotion] for emotion in val_dataset['emotion']]
test_labels = [emotion_to_num[emotion] for emotion in test_dataset['emotion']]

# Convert input data and labels to TensorFlow tensors
train_inputs_tf = {'input_ids': tf.convert_to_tensor(train_inputs['input_ids']),
                   'attention_mask': tf.convert_to_tensor(train_inputs['attention_mask'])}
#val_inputs_tf = {'input_ids': tf.convert_to_tensor(val_inputs['input_ids']), 'attention_mask': tf.convert_to_tensor(val_inputs['attention_mask'])}
test_inputs_tf = {'input_ids': tf.convert_to_tensor(test_inputs['input_ids']),
                  'attention_mask': tf.convert_to_tensor(test_inputs['attention_mask'])}
train_labels_tf = tf.convert_to_tensor(train_labels)
#val_labels_tf = tf.convert_to_tensor(val_labels)
test_labels_tf = tf.convert_to_tensor(test_labels)

In [ ]:
#Freezing the model weights before training
for layer in tf_model.layers:
    layer.trainable = False

#Unfreezing some of the model layers to be trained
num_of_trainable_layers = 4
for layer in tf_model.roberta.encoder.layer [-num_of_trainable_layers:]:
    layer.trainable = True

In [ ]:
#Defining a new classificatoin layer to be added to the end of the model
num_emotions = train_dataset['emotion'].value_counts ().count()  #num_emotions = 6
num_unit_1 = 256
num_unit_2 = 256
layer_1 = tf.keras.layers.Dense ( num_unit_1, activation = "relu", name = 'fully_connected_layer1' )
#layer_2 = tf.keras.layers.Dense (num_unit_2, activation = "relu", name = 'fully_connected_layer2')
classification_layer = tf.keras.layers.Dense ( num_emotions , activation='softmax', name='arb_classification_layer' )

In [ ]:
#Inputs and Attention mask layers
input_ids = tf.keras.Input (shape =(None,), dtype=tf.int32, name = 'input_ids')
attention_mask_input = tf.keras.Input (shape= (None,), dtype=tf.int32, name = 'attention_mask')

#inputs = tf.convert_to_tensor(input_ids)
#attention_mask = tf.convert_to_tensor(attention_mask_input)

# Pass the input through the pre-trained model layers
#outputs = tf_model({'input_ids': input_ids, 'attention_mask': attention_mask_input})
outputs = tf_model.bert ({'input_ids': input_ids, 'attention_mask': attention_mask_input})

#Extracting the pooled output
pooled_output = outputs.last_hidden_state [:, 0, :]

#Passing the model output through the classification layer
classification_output = classification_layer ( layer_1 (pooled_output))

#Building the final model
final_model = tf.keras.Model ( inputs = [input_ids, attention_mask_input], outputs = classification_output )

In [ ]:
#Compile the model
lr_rate = 2e-4
optimizer = tf.keras.optimizers.Adam ( learning_rate = lr_rate )
loss = tf.keras.losses.SparseCategoricalCrossentropy ()
metrics = ['accuracy']
final_model.compile ( optimizer = optimizer, loss= loss, metrics= metrics )

In [ ]:
#final model architecture
final_model.summary ()

In [ ]:
#Load an existing model
final_model = tf.keras.models.load_model ( 'D:/Users/chatbot_emotions_detection/artifacts/experiments_models/arabic_bert_final_model', custom_objects={'TFXLMRobertaForSequenceClassification': TFXLMRobertaForSequenceClassification} )
final_model.layers[2].trainable = False
#final_model.layers[3].trainable = False
#Adding new classification layer with 6 outputs to our model 
"""outputs = final_model.output
arb_classification_layer = classification_layer ( outputs )
arb_final_model = tf.keras.Model (inputs = final_model.input, outputs = arb_classification_layer)"""
final_model.summary ()

In [ ]:
from transformers import TFAutoModelForSequenceClassification

In [ ]:
tf.keras.models.load_model ( 'D:/Users/chatbot_emotions_detection/artifacts/experiments_models/arabic_bert_final_model',
                            custom_objects = { 'TFAutoModelForSequenceClassification': TFAutoModelForSequenceClassification } )

In [ ]:
#Training the model
num_epochs = 1
history = final_model.fit(
    {'input_ids': train_inputs_tf ['input_ids'], 'attention_mask': train_inputs_tf['attention_mask']},
    train_labels_tf,
    validation_data=({'input_ids': test_inputs_tf['input_ids'], 'attention_mask': test_inputs_tf['attention_mask']}, test_labels_tf),
    batch_size=8,
    epochs= num_epochs
)

In [ ]:
#Let's compute the F1 score
y_predicted_prob = test_model.predict ( test_inputs_tf )
y_predicted = np.argmax ( y_predicted_prob, axis = 1 )

#Computing F1 score and accuracy
F1_score = f1_score ( test_labels, y_predicted, average = 'macro')
Accuracy_score = accuracy_score ( test_labels, y_predicted )

#Precision and recall
precision = precision_score (test_labels, y_predicted, average = 'macro')
recall = recall_score (test_labels, y_predicted, average= 'macro')

print ( f"F1 score on test data : {F1_score}")
print ( f"Accuracy score on test data: {Accuracy_score}")
print ( f"Precision score on test data: {precision}")
print ( f"Recall score on test data: {recall}")

In [ ]:
#Storing metrics values
shot = 23
shot = shot+1
accuracy_historical = {}
accuracy_historical[f'shot{shot}'] = {'training size': train_dataset.shape [0],
                                      'added layers': 2,
                                      'num unit': num_unit_1,
                                      'epochs': 3,
                                      'learning rate': lr_rate,
                                      'accuracy': Accuracy_score,
                                      'f1 score': F1_score,
                                       'precision': precision,
                                        'recall': recall }

#Storing the accuracy historincal in our local
with open ( 'D:/Users/chatbot_emotions_detection/artifacts/accuracy_historical_arb.txt', 'a' ) as file:
    for key, value in accuracy_historical.items():
        file.write ( f'{key} : {value} \n' )

In [ ]:
#Let's story history results in case we want to retrain the model multiplt times
#history_ = {'loss': [], 'val_loss': []}
try : 
    history_['loss'] += history.history['loss']
    history_['val_loss'] += history.history['val_loss']
except:
    history_ = {}
    history_['loss'] = history.history['loss']
    history_['val_loss'] = history.history['val_loss']

In [ ]:
#Ploting the loss
plt.plot ( range (len (history_['loss'])), history_['loss'] )
plt.plot ( range (len (history_['val_loss'])), history_['val_loss'] )
plt.xlabel ( 'Epochs' )
plt.ylabel ('loss')
plt.legend ( ['loss', 'vlaidation loss'] )

In [ ]:
#Storing the model
final_model.save ( "D:/Users/chatbot_emotions_detection/artifacts/experiments_models/arabic_bert_local" )

#### Inferences

In [36]:
import tensorflow as tf

In [ ]:
#Function that predict emotion from a text
def predict_emotion ( text: list ):
    if type(text) != list:
        text = [text]

    tokenized_text = tokenizer ( text, padding = True, truncation= True, max_length = 500, return_tensors = 'tf' )
    tokenized_text = { 'input_ids': tokenized_text["input_ids"],
                    'attention_mask': tokenized_text["attention_mask"] }
    predicted_emotions_probabilitie = test_model.predict ( tokenized_text )
    #predicted_emotion = predicted_emotions_probabilitie
    emotion =  list (predicted_emotions_probabilitie.argmax( axis = 1 ))
    for i in range ( len(text) ):
        print ( f"Emotoin of text : {text[i]} is << {num_to_emotion [emotion[i]]} >>" ) 
